In [25]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
import glob
import xml.etree.ElementTree as ET

from data_generator.object_detection_2d_data_generator import DataGenerator

In [26]:


#входное разрешение модели
img_height = 320 # Height of the input images
img_width = 544 #Width of the input images

In [27]:
"""
Структура папок:
base_folder
    obj_name1
        folder1
            Images_xml
        folder2
            Images_xml
        ...
    obj_name1
        folder1
            Images_xml
        folder2
            Images_xml
        ...
        
"""

def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for folder in os.listdir(path):
        print(folder, len(glob.glob(f'{path}/{folder}/Images_xml/*.xml')))
        for xml_file in glob.glob(f'{path}/{folder}/Images_xml/*.xml'):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            filename = root.find('filename').text
            filename = f'{path}/{folder}/Images_xml/{filename}'
#             print(filename)
            if root.find('size') is None:
                img = cv2.imread(filename)
                height, width,_ = img.shape
            else:
                width = int(root.find('size').find('width').text)
                height = int(root.find('size').find('height').text)
            for member in root.findall('object'):
                bndbox = member.find('bndbox')
                value = (filename,
                         width,
                         height,
                         member.find('name').text,
                         int(bndbox.find('xmin').text),
                         int(bndbox.find('ymin').text),
                         int(bndbox.find('xmax').text),
                         int(bndbox.find('ymax').text),
                         )
                xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [28]:
#Создаем DataFrame из всех имеющихся изображений
base_path = 'W:/Work/data/DATASETS/POLARIS_PASS' #путь к основной папке
df = pd.DataFrame()
for obj_folder in os.listdir(base_path):
    print(obj_folder)
    df = df.append(xml_to_csv(os.path.join(base_path, obj_folder)))
#Для создания датасета из obj_folder
# df = xml_to_csv('W:/Work/data/DATASETS/POLARIS')

Camera_left
05_09 289
1 1087
2 157
done_01_09_2022 95
edit_02_09 192


C:\Users\WorkStation\AppData\Local\Temp\ipykernel_19152\2375333048.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(xml_to_csv(os.path.join(base_path, obj_folder)))


In [29]:
df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,1178,118,1227,337
1,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,1226,224,1291,372
2,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,525,34,580,173
3,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,847,109,915,400
4,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,417,165,482,428
...,...,...,...,...,...,...,...,...
6914,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,875,56,924,194
6915,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,920,61,964,212
6916,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,471,70,536,280
6917,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,1408,640,person,534,19,585,147


In [30]:
#Преобразуем координаты рамок к нужному размеру
labels_to_int = {'person':1, 'staff':1}
# df = df[df['class'] == 'car']
df = pd.DataFrame({'image_name':df['filename'],
                   'xmin':(df['xmin']/df['width']*img_width).astype(int),
                   'xmax':(df['xmax']/df['width']*img_width).astype(int),
                   'ymin':(df['ymin']/df['height']*img_height).astype(int),
                   'ymax':(df['ymax']/df['height']*img_height).astype(int),
                   'class_id':[labels_to_int[label] for label in df['class']]})

df.to_csv('data/bukmeker288.csv', index=False)
df

,image_name,xmin,xmax,ymin,ymax,class_id
0,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,455,474,59,168,1
1,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,473,498,112,186,1
2,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,202,224,17,86,1
3,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,327,353,54,200,1
4,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,161,186,82,214,1
...,...,...,...,...,...,...
6914,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,338,357,28,97,1
6915,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,355,372,30,106,1
6916,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,181,207,35,140,1
6917,W:/Work/data/DATASETS/POLARIS_PASS\Camera_left...,206,226,9,73,1


In [16]:
len(pd.unique(df['image_name']))

4031

In [31]:
#Создаем файл датасета для обучения детектора
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
images_dir = ''

train_labels_filename = 'data/bukmeker288.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

train_dataset.create_hdf5_dataset(file_path='dataset_polaris_pass320.h5',
                                  resize=[img_height,img_width],
                                  variable_image_size=True,
                                  verbose=True)

Creating HDF5 dataset: 100%|███████████████████████████████████████████████████████| 1820/1820 [00:22<00:00, 81.88it/s]
